In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import time
from numba import jit
import pickle
from funcoes_BuscaLocal import *
from funcoes_GA import *
from funcoes_gerais import *

dados = {}

with open("dados.pkl", "rb") as infile:
    dados = pickle.load(infile)

solucoes_pandas = pd.read_pickle('solucoes.pkl')
solucoes = solucoes_pandas.to_dict()
solucoes_GA = {}

lista_hs = [0.8, 0.6, 0.4, 0.2]
#conjuntos = [10,20,50,100,200,500,1000]
conjuntos = [1000]
lista_z = [0.25 , 0.5 , 0.6 , 0.75, 2]
#lista_problemas = list(range(1,11))
lista_problemas = [5]

populacao = {}
qtd_pop_inicial = {}
















In [ ]:
for conjunto in conjuntos:
    for h in lista_hs:
        for problema in lista_problemas:
            breakpoint()
            inicio=time.time()
            pi = np.array(dados[conjunto][problema]['pi'])
            ai = np.array(dados[conjunto][problema]['ai'])
            bi = np.array(dados[conjunto][problema]['bi'])

            d=int(sum(pi)*h)
            
            #carreguei as soluções da heurística construtiva e de melhoria
            pop_sols = np.array([solucoes[(c,p,hi,z_corte)] for (c,p,hi,z_corte) in solucoes if conjunto == c and problema == p and h == hi])
            pop_objs = np.array([])
            #calculei os objetivos
            for individuo in pop_sols:
                pop_objs = np.append(pop_objs,calcula_objetivo_sr(individuo, ai, bi, pi, d))
            
            diversidade = calcula_diversidade(pop_sols,pop_objs)

            #Nesse ponto posso definir os parâmetros.
            #foi mantido aqui pois o valor utilizado
            #pode ser uma função da dimensão do problema, por exemplo.
            
            n_iter_ga = 500
            taxa_mutacao_inicial = 0          
            max_pop = 500
            
            
            taxa_mutacao = taxa_mutacao_inicial
            obj_original = min(pop_objs)
            for iteracao in range(n_iter_ga):
                
                print(conjunto, h, problema, iteracao ,len(pop_sols), round(diversidade,2), obj_original, min(pop_objs))
                
                #gero novas soluções por cruzamentos e mutações;
                #a lista nova_solucoes é 2x maior que a população atual
                novas_solucoes = expande_populacao(pop_sols,pop_objs,0.05)
                
                #avaliando objetivo das novas soluções
                novas_solucoes_objs = avaliar_pop_sr(novas_solucoes,ai,bi,pi,d)

                #reparando soluções novas infactíveis
                for cada_infactivel in np.where(novas_solucoes_objs==1e15)[0]:
                    set_E,set_T = transforma(novas_solucoes[cada_infactivel])
                    set_E,set_T = repara_solucao(set_E,set_T,ai,bi,pi,d)
                    nova_sol = transforma_bin(set_E,set_T)
                    novas_solucoes[cada_infactivel] = nova_sol
                    novas_solucoes_objs[cada_infactivel] = calcula_objetivo_sr(nova_sol,ai,bi,pi,d)

                #insiro novas soluções na população
                pop_sols = np.vstack((pop_sols,novas_solucoes))
                pop_objs = np.append(pop_objs,novas_solucoes_objs)
                
                #agora vou avaliar a população e matar alguns indivíduos
                #se a população estiver maior que meu parâmetro max_pop
                
                chora_nene = oprime_fracos(pop_objs,max_pop)
                pop_sols = pop_sols[chora_nene==True]
                pop_objs = pop_objs[chora_nene==True]
                diversidade = calcula_diversidade(pop_sols,pop_objs)
            print(conjunto, h, problema, iteracao ,len(pop_sols), round(diversidade,2), obj_original, min(pop_objs))

                    
                    
            fim = time.time()





